# SAP TechEd 2025, session DA261
## Unlocking AI-driven insights from your business data in SAP HANA Cloud

## Exercise 0 - Getting started

### SAP Business Application Studio - Python environment setup and configuration 

- Install required Python modules
- Restart Python kernel
- Prepare the SAP HANA Cloud connection credential file
- Validate SAP HANA Cloud connectivity



### Installation of Python modules

#### Package overview
The following Python modules are to be installed during this hands-on introduction. 

__hdbcli__
The Python Database API Specification v2.0 (PEP 249) defines a set of methods that provides a consistent database interface independent of the actual database being used. The Python extension module for SAP HANA implements PEP 249. Once you install the module, you can access and change the information in SAP HANA databases from Python.

__hana-ml__
The Python Machine Learning client for SAP HANA package enables Python data scientists to access SAP HANA data using SAP HANA dataframes and utilize SAP HANA's database AI engine algorithm libraries (PAL and APL) and NLP services to build and apply machine learning models within the database natively. While scripting in python, SQL is generated on the fly and executed in the SAP HANA database. The SAP HANA dataframe is simply a SQL select statement against any data in the SAP HANA database, data is not copied into the python environment, unless explicitly collected.

<br>


</br>

#### Install Python packages

Run the following package installations. **pip** is the package installer for Python. You can use pip to install packages from the Python Package Index and other indexes.

In [1]:
!pip install hdbcli --break-system-packages
!pip install hana-ml --break-system-packages
#!pip install generative-ai-hub-sdk[all] --break-system-packages
!pip install folium --break-system-packages
!pip install ipywidgets --break-system-packages

# kernel restart required!!!



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


#### Restart Python kernel ...
...

> **Note:** Jupyter Notebook kernel restart required after package installation.

#### Restart Python kernel

The Python kernel needs to be restarted before continuing. 
![Set user](./images/BAS_python_env_restart.png)


> **Note** This may take a minute.

### Connect to the HANA Cloud

The provided Python script imports database connection modules and initiates a connection to a SAP HANA Cloud instance using ...
-  the connection details are edited within a user.ini file and reused for creating the connection from python in each exercise
-  Alternatively, the user is prompted to enter hostname, their username and password, which are then used to establish a secure connection to the SAP HANA Cloud database. 

</br>

> **Note** Use your username and password supplied to logon to the SAP HANA Cloud database. Find the **host_address** in the lesson content.

## Load the user connection data and connect to the SAP HANA database instance

👉🏻 **Before running the next cell** make sure:

1. The content of the [./ex0/temp_user.ini](ex0/temp_user.ini) file is copied to a file [./ex0/user.ini](ex0/user.ini) and 
2. the user and the password given to you by the instructor are updated in `user.ini` file 

👆🏻

👇🏼  
![Set user](./images/BAS_python_edit_userini.png)

In [2]:
import os, sys

file_path = './user.ini'  # Replace this with the path to your file

if os.path.exists(file_path):
    print(f"The file '{file_path}' exists.")
    from hana_ml.algorithms.pal.utility import Settings
    myhost, myport, myuser, mypwd = Settings.load_config(file_path)
    if myhost == '' or myport == 0 or myuser == '' or mypwd == '':
        print("At least one required connection parameter is empty.")
    else:
        print(f"Connection to SAP HANA db at {myhost}:{myport} \nwill be attempted for the user {myuser}")
else:
    sys.exit(f"The file '{file_path}' does not exist! Create and configure it first.")
    

SystemExit: The file './user.ini' does not exist! Create and configure it first.

C:\Users\D059078\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
from hana_ml import dataframe as hdf
myconn=hdf.ConnectionContext(
    address=myhost, 
    port=myport, 
    user=myuser,
    password=mypwd
)
print(f"Connected to SAP HANA db version {myconn.hana_version()} \nat {myhost}:{myport} as user {myuser}")

In [ ]:
print(f'Current time on the SAP HANA server: {myconn.sql("SELECT NOW() FROM DUMMY").collect().CURRENT_TIMESTAMP[0]}')

In [ ]:
# Alternative 1 - Check connection using prompted connection credentials

from hdbcli import dbapi
#from langchain_community.vectorstores.hanavector import HanaDB

# 13b7c15d-848f-40b5-9259-c9c36ab85f56.hna1.prod-eu10.hanacloud.ondemand.com

host_address = input("Enter HANA Cloud Hostname")
hdb_user = input("Enter Username")
hdb_password = input("Enter Password :")

connection = dbapi.connect(
    host_address,
    port="443",
    user=hdb_user,
    password=hdb_password,
    autocommit=True,
    sslValidateCertificate=False,
)

print(connection.connection.isconnected())
print(connection.hana_version())

In [ ]:
# Alternative 2b - The HANA Dataframe ConnectionContext class offers various ways of connecting to HANA Cloud, here we re-use promted credentials from the step before
# Create the SAP HANA CLoud connection
import hana_ml.dataframe as dataframe
from hana_ml.dataframe import ConnectionContext
#cc = dataframe.ConnectionContext( address="<hana-cloud-hostname>", port=443,  user="<HANA-user>")
#cc = dataframe.ConnectionContext( userkey=<your-key-name>, sslValidateCertificate=False)

# Utilize connection credentials from external file
from hana_ml.algorithms.pal.utility import Settings
#url, port, user, pwd = Settings.load_config('./0-setup/HC_trial_PAL.ini')
cc = ConnectionContext(host_address, 443, hdb_user, hdb_password)

# Utilize connection keys from the hdbclient connection store
#cc = dataframe.ConnectionContext(userkey='<MYHCTRIALkeyexample>', sslValidateCertificate=False)
#cc = ConnectionContext(userkey='HC24Q1_VECENG')

# Control connection
print(cc.connection.isconnected())
print(cc.hana_version())